# <span style="color: teal">Data Crawler</span>

### Instalando Pacotes

In [ ]:
%pip install pandas
%pip install pyyaml

### Obtendo Dados da Fonte

In [ ]:
import yaml
import pandas as pd
import urllib.parse

In [ ]:
# Parâmetros da requisição
parameters = {"dataTransacaoInicio": "01/01/2019", "dataTransacaoFim": "31/12/2019"} 
query_string = urllib.parse.urlencode(parameters)
#query_string

In [ ]:
# Lendo as configurações
with open('.\configurations.yml') as file:
    configurations = yaml.load(file, Loader=yaml.FullLoader)
    #print(configurations)

In [ ]:
df = pd.DataFrame()

def get_data_from_source():
    """Collect data from data source."""
    page = 0
    df = pd.DataFrame()
    while True:
        try:
            url = F"{configurations['DataSource']['Host']}{configurations['DataSource']['Route']}?{query_string}&pagina=" + str(page+1)
            api_response = pd.read_json(url)
            
            if page == 0:
                df = api_response # create new dataframe
            else:
                df = pd.concat([df, api_response]) # concat received data to created dataframe
                
            milestone(page)
            page += 1
        except:
             break
    return df

def milestone(page:int, mark=1000):
    if page == 0:
        print("- Crawling Iniciado -")
        return
        
    if page % mark == 0: 
        print('{} páginas lidas'.format(page))

In [ ]:
def data_transformation(df):
    #changes de valorTransacao to float
    df['valorTransacao'] = df['valorTransacao'].str.replace(' ','').str.replace('.','').str.replace(',','.').astype(float)
    #reset index
    df.reset_index(drop=True, inplace=True)
    #break columns of dicts into actual columns and merge them into the original df by index
    df = df.merge(pd.DataFrame(list(df['tipoCartao'])),how='left', left_index=True, right_index=True)
    df.rename(columns={'id_x' : 'id','id_y' : 'id_tipoCartao','codigo' : 'codigo_tipoCartao','descricao' : 'descricao_tipoCartao'},inplace=True)
    df = df.merge(pd.DataFrame(list(df['estabelecimento'])),how='left', left_index=True, right_index=True)
    df.rename(columns={'nome': 'nome_estabelecimento', 'codigoFormatado':'codigoFormatado_estabelecimento'},inplace=True)
    df = df.merge(pd.DataFrame(list(df['cnae'])),how='left', left_index=True, right_index=True)
    df = df.merge(pd.DataFrame(list(df['municipio'])),how='left', left_index=True, right_index=True)
    df = df.merge(pd.DataFrame(list(df['naturezaJuridica'])),how='left', left_index=True, right_index=True)
    df = df.merge(pd.DataFrame(list(df['uf'])),how='left', left_index=True, right_index=True)
    df.rename(columns={'nome': 'nome_uf','codigo' : 'codigo_estabelecimento'},inplace=True)
    df = df.merge(pd.DataFrame(list(df['unidadeGestora'])),how='left', left_index=True, right_index=True)
    df.rename(columns={'nome' : 'nome_unidadeGestora','codigo' : 'codigo_unidadeGestora', 'sigla':'sigla_uf'},inplace=True)
    df = df.merge(pd.DataFrame(list(df['orgaoVinculado'])),how='left', left_index=True, right_index=True)
    df.rename(columns={'nome' : 'nome_orgaoVinculado', 'sigla':'sigla_orgaoVinculado'},inplace=True)
    df = df.merge(pd.DataFrame(list(df['orgaoMaximo'])),how='left', left_index=True, right_index=True)
    df.rename(columns={'codigo' : 'codigo_orgaoMaximo', 'sigla':'sigla_orgaoMaximo', 'nome':'nome_orgaoMaximo'},inplace=True)
    df = df.merge(pd.DataFrame(list(df['portador'])),how='left', left_index=True, right_index=True)
    df.rename(columns={'nome': 'nome_portador', 'codigoFormatado':'codigoFormatado_portador'},inplace=True)
    #delete columns of dicts
    df.drop(['estabelecimento', 'cnae', 'municipio', 'naturezaJuridica', 'uf','tipoCartao','unidadeGestora','portador', 'orgaoVinculado', 'orgaoMaximo'], axis=1, inplace=True)
    return df

In [ ]:
df = data_transformation(get_data_from_source())
#df.head()

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///database.db')
df.to_sql(name='cartoes',con=engine,if_exists='append')